# PuLP VS SciPy: Which optimization method is better?

In this artifact, we'll be exploring which of the two different packages and methods used for linear optimization problems is more efficient and is "better" to use. We will be looking at runtime, ease of writing code, and flexibility amongst some other factors. Firstly, we'll need to import the necessary packages.

In [8]:
import pulp
from pulp import *
from scipy.optimize import linprog
import numpy as np
import IPython
import timeit

To begin with, we will look at how each package deals with linear optimization and how the problems are structured.

### PuLP
In PuLP, we create the variables and constraints separately for each constraints. Once the constraints, objective function and variables are defined, we create a problem using the `LpMinimize` or `LpMaximize` function depending on what we are aiming to do, and then add the variables, constraints and function to the problem. In PuLP, we can add each separate constraint and allow it to be an inequality or equality constraint depending on the problems. PuLP also has multiple different types of linear solvers, but for this specific example, we will be using the GLPK solver, which is generally the default on PuLP.



### SciPy
In SciPy, the primary function used for linear optimization is the `linprog()` function. `linprog()` takes in the constraints in its vector and matrix form as the input and solves the problem. To differentiate between equality and inequality constraints, `linprog()` has different arguments which allow us to specify what type of constraint we have. The main thing to note when using SciPy to solve a linear optimization problem is that `linprog()` always defaults to minimize, so if we are solving a problem where we are looking to maximize the objective function, we would have to input the negative of the objective function and take the negative of the solution as our final answer.


## Sample Problem

This sample problem was taken from the Vanderbei Textbook. The problem is as such:

$$
\begin{align}
\text{max } & \mathbf{c}^{T}\mathbf{x} \\
\text{subject to: } & \mathbf{A}\mathbf{x} \leq \mathbf{b}
\end{align}
$$
where
$$
\mathbf{A} = \begin{pmatrix} 2 & 3 & 1 \\ 4 & 1 & 2\\ 3 & 4 & 2 \end{pmatrix}    
\mathbf{c} = \begin{pmatrix} 5 \\ 4 \\ 3 \end{pmatrix} 
\mathbf{b} = \begin{pmatrix} 5 \\ 11 \\ 8 \end{pmatrix} 
$$

### PuLP construction
The next few cells construct the problem above in PuLP. Note that within PuLP, there are multiple different ways to construct the problem and this is just one of them. First, we create our variables, and then create dictionaries to represent our constraints and objective function. The next step is to then define the problem. As the variables are defined in a list form, we use `LpVariable.dicts` to convert the list into a dictionary of variables as we need, and set the lower bound to 0. Then we can go ahead and add our objective function and constraints to the problem, and then solve. We also look at the status of the solution to make sure its optimal and its what we are looking for.

In [11]:
%%time

Variables = ['x1', 'x2', 'x3']

# the constraints as in the A matrix

a1 = {'x1': 2,
      'x2': 3,
      'x3' : 1}

a2 = {'x1': 4,
      'x2': 1,
      'x3' : 2}

a3 = {'x1': 3,
      'x2': 4,
      'x3' : 2}

# the objective function
c = {'x1': 5,
    'x2': 4,
    'x3' : 3}

problem = LpProblem("Problem1", LpMaximize)
problem_vars = LpVariable.dicts('x', Variables, lowBound = 0)
# adding the constraints
problem += lpSum([c[i] * problem_vars[i] for i in Variables])
problem += lpSum([a1[i] * problem_vars[i] for i in Variables]) <= 5, "constraint 1"
problem += lpSum([a2[i] * problem_vars[i] for i in Variables]) <= 11, "constraint 2"
problem += lpSum([a3[i] * problem_vars[i] for i in Variables]) <= 8, "constraint 3"
problem.solve()

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --cpxlp /tmp/64f3c6ecf833405b8dd9765ed5628395-pulp.lp -o /tmp/64f3c6ecf833405b8dd9765ed5628395-pulp.sol
Reading problem data from '/tmp/64f3c6ecf833405b8dd9765ed5628395-pulp.lp'...
3 rows, 3 columns, 9 non-zeros
8 lines were read
GLPK Simplex Optimizer, v4.65
3 rows, 3 columns, 9 non-zeros
Preprocessing...
3 rows, 3 columns, 9 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  4.000e+00  ratio =  4.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 3
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (3)
*     2: obj =   1.300000000e+01 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (39693 bytes)
Writing basic solution to '/tmp/64f3c6ecf833405b8dd9765ed5628395-pulp.sol'...
CPU times: user 4.74 ms, sys: 16.7 ms, total: 21.4 ms
Wall time: 30.3 ms


1

In [12]:
print(LpStatus[problem.status])
for i in problem.variables():
    print("Variable {0} = {1}".format(i.name, i.varValue))
print("z = %s"%(value(problem.objective)))

Optimal
Variable x_x1 = 2.0
Variable x_x2 = 0.0
Variable x_x3 = 1.0
z = 13.0


### SciPy construction

In SciPy, as said before, all we need is the matrix and vector inputs, and then we can put it into the `linprog()` function. Here, we will use `numpy` to create the matrix and vectors.

In [14]:
%%time

A = np.array([[2., 3., 1.], [4., 1., 2.], [3., 4., 2.]])
c = np.array([5., 4., 3.])
b = np.array([5., 11., 8.])
linprog(-c, A, b)

CPU times: user 2.55 ms, sys: 2.76 ms, total: 5.31 ms
Wall time: 4.95 ms


           con: array([], dtype=float64)
 crossover_nit: 0
         eqlin:  marginals: array([], dtype=float64)
  residual: array([], dtype=float64)
           fun: -13.0
       ineqlin:  marginals: array([-1., -0., -1.])
  residual: array([0., 1., 0.])
         lower:  marginals: array([0., 3., 0.])
  residual: array([2., 0., 1.])
       message: 'Optimization terminated successfully. (HiGHS Status 7: Optimal)'
           nit: 2
         slack: array([0., 1., 0.])
        status: 0
       success: True
         upper:  marginals: array([0., 0., 0.])
  residual: array([inf, inf, inf])
             x: array([2., 0., 1.])

## Discussion

The wall time taken for the SciPy function to run is significantly shorter than that of PuLP. But before we dive in to what I found, we'll take a look at what generally is accepted to be the better optimization tool.

### Previously Existing Comparisons
When doing this artifact, I found that the main comparisons people had already made between the two packages are as follows:
- SciPy does not provide classes and functions from which we can build a model, but is based off of vectors and matrices instead. This can be tedious for larger optimization problems. 
- PuLP has a range of different solvers we can use, which provide more flexibility
- SciPy only allows minimization problems
- Defining constraints through SciPy is rigid, and multiple different vectors need to be stated if there are both inequality and equality constraints
- SciPy is quicker as it uses the HiGHS solver

### Results
From this artifact, I found that much of what was previously said rings true, but I find that the two different packages would be equally benefitial for different cases. In solving simple LP problems like the above one, I found that SciPy is much more effective and is easier to use, whereas PuLP takes more time in building the constraints and is more complex. When building larger models, I think PuLP would be better to use as it has more flexibility with the constraints, and allows for more complex problems to be defined. Overall, even with SciPy having a faster runtime, I find that PuLP is better to use in general.

### Further exploration
While these arethe two common methods used for linear optimization, there are also other packages, such as Pyomo and Google OR tools, that can be used to solve such problems. This provides more options and further techniques to explore, and see which one is "better".

# References
Real Python. (2020, November 7). Hands-on Linear Programming: Optimization with python. Real Python. Retrieved December 8, 2022, from https://realpython.com/linear-programming-python/ 